# [George McNinch](http://gmcninch.math.tufts.edu) Math 87 - Spring 2025

# Week 15

# Singular value decomposition and null spaces

# Why do we need to find the null space?

We have seen how to use `numpy` to solve matrix equations -- i.e. systems of linear equations -- in certain cases. 

For non-square coefficient matrices, the main available tool is the least squares approximation.

In [19]:
import numpy as np
import numpy.linalg as la

A = np.array([[1,1,1,1], [1,2,1,2]])
b = np.array([1, 1])


try:
    la.solve(A,b) 
except la.LinAlgError:
    print(f"error: matrix\n{A}\nis not square.")

error: matrix
[[1 1 1 1]
 [1 2 1 2]]
is not square.


In [13]:
la.lstsq(A,b)

(array([5.00000000e-01, 2.22044605e-16, 5.00000000e-01, 2.22044605e-16]),
 array([], dtype=float64),
 np.int32(2),
 array([3.70245917, 0.54018151]))

Least squares gives (roughly) the solution `[1/2, 0, 1/2, 0]` (which is indeed a solution).

But also `[1,0,0,0]` and `[0,0,1,0]` are solutions. How do we convince `python` to find these solutions for us??!!

Well, the point really is that we should find the null space `N(A)` of `A`. And then every solution to `Ax=b` has the form
`p+n` where `p` is a fixed particular solution to the equation, and `n` is any solution to the homogeneous equation `Ax=0` --i.e. `n` is in the *null space* `N(A)` of `A`.

In math70, you'd solve this problem by performing row operations on the matrix `A`.

For example, we can subtract the first row from the second:

In [21]:
A1 = np.array([[1,0],[-1,1]]) @ A
A1

array([[1, 1, 1, 1],
       [0, 1, 0, 1]])

In [23]:
# now subtract second row from first
A2 = np.array([[1,-1],[0,1]]) @ A1
A2

array([[1, 0, 1, 0],
       [0, 1, 0, 1]])

Now the matrix is in echelon form and we see that its null space is spanned by the vectors
`[1,0,-1,0]` and `[0,1,0,-1]`.

And we see that the general solution to `Ax = [1,1]` is given by

`[1,0,0,0] + s[1,0,-1,0] + t[0,1,0,-1] = [1 + s, t , -s, -t]`

But how to automate finding the null space?

# Singular value decomposition, quickly.

The best computational answer is to compute the so called singular value decomposition of the matrix, which makes it easy to find many things, including the null space.

So, let `A` be an `m x n` matrix (with real or even complex coefficients).

Write `A*` for the *conjugate transpose* of `A`. In numpy this is `A.H` if `A` is created via `np.matrix`. If `A` is created by `np.array` you'll need to instead use
`A.conj().T` ("conjugate transpose").

In [43]:
Ac = np.matrix([[1+1j,1,1],[1,2+1j,-1]])
print(Ac)

[[ 1.+1.j  1.+0.j  1.+0.j]
 [ 1.+0.j  2.+1.j -1.+0.j]]


In [44]:
print(Ac.T)
print(Ac.H)

[[ 1.+1.j  1.+0.j]
 [ 1.+0.j  2.+1.j]
 [ 1.+0.j -1.+0.j]]
[[ 1.-1.j  1.-0.j]
 [ 1.-0.j  2.-1.j]
 [ 1.-0.j -1.-0.j]]


In [46]:
print(Ac.H @ Ac)

[[ 3.+0.j  3.+0.j  0.-1.j]
 [ 3.+0.j  6.+0.j -1.+1.j]
 [ 0.+1.j -1.-1.j  2.+0.j]]


**Theorem** 
:   The matrices `A.H @ A` and `A @ A.H` -- i.e. $A^* \cdot A$ or $A \cdot A^*$ -- have the same rank and the same non-zero eigenvalues. These non-zero eigenvalues are    
    positive real numbers. Write $$\sigma_1^2 \ge \sigma_2^2 \ge \cdots \ge \sigma_\rho^2 > 0$$ for the positive eigenvalues; these numbers $\sigma_i$ are known as the *singular 
    values* of the matrix `A`.

Let $S = \operatorname{diag}(\sigma_1,\cdots,\sigma_\rho)$ be the diagonal matrix with diagonal entries the $\sigma_i$ and let
$\Sigma$ be the block `m x n` matrix
$$\Sigma = \begin{pmatrix} S & 0 \\
0 & 0 \end{pmatrix}$$

**Theorem** (SVD)
There is an `m x m` matrix $U$ and an `n x n` matrix $V$ such that  
- $A = U \cdot \Sigma \cdot V^T$
- The columns of $U$ form an orthonormal basis of $\mathbf{C}^m$ and the columns of $V$ form an orthonormal basis of $\mathbf{C}^n$.
- If $A$ is a matrix with real coeffs, then $U$ and $V$ can be taken to be real matrices whose columns are orthonomal bases of $\mathbf{R}^m$ resp. $\mathbf{R}^n$.

With notation as in the Theorem, write $v_1,\cdots,v_n$ for the columns of the matrix $V$. Then $v_{\rho + 1},\cdots,v_{n}$ form a basis for the null space of $A$.

Moreover, for $1 \le i \le \rho$, $A v_i = \sigma_i u_i$.

An important tool in finding the `SVD` is the Grahm-Schmidt orthogonalization process.

There is a `numpy` algorithm for computing the singular value decomposition of a matrix $A$, from which you can immediately determine a basis for the null space of $A$.

Let's try this for the matrix `A` we considered above. Recall that:

In [47]:
A = np.array([[1,1,1,1], [1,2,1,2]])

b = np.array([1, 1])

In [48]:
U,Sigma,Vt = la.svd(A)

Now we have `A = U @ Sigma @ Vt`, and the last few columns of `Vt` form a basis for the null space of $A$. It just remains to determine the dimension of the null space,
which in this case is just $n - \operatorname{rank}(A) = 4 - \operatorname{rank}(A)$.

In [56]:
nd = 4 - la.matrix_rank(A)
nd

np.int64(2)

We see that there are two vectors in the null space, and we can extract them as follows:

In [60]:
null = [ V.T[:,j] for j in range(nd,4) ]
null

[array([-0.7,  0.1,  0.7, -0.1]), array([-0.1, -0.7,  0.1,  0.7])]

In [62]:
[ A @ n for n in null ]

[array([8.32667268e-17, 5.55111512e-17]),
 array([ 0.00000000e+00, -1.11022302e-16])]

Note that the vectors in `null` really are orthonormal:

In [67]:
[ [float(null[i]@null[j]) for i in range(2)] for j in range(2) ]

[[1.0, -1.3877787807814457e-17], [-1.3877787807814457e-17, 1.0]]

Recall that we pointed out that `[1,0,-1,0]` and `[0,1,0,-1]` formed a(nother) basis for the null space. We can express these vectors in terms of the new basis `null`, using orthonormality.

In [77]:
b1 = np.array([1,0,-1,0])
b2 = np.array([0,1,0,-1])

def proj_to_null(v):
    return sum([(null[i] @ v) * null[i] for i in range(2)])

In [78]:
proj_to_null(b1)

array([ 1.00000000e+00,  0.00000000e+00, -1.00000000e+00,  2.77555756e-17])

In [79]:
proj_to_null(b2)

array([-8.32667268e-17,  1.00000000e+00, -2.77555756e-17, -1.00000000e+00])